# Redfish Ansible using builtin URI method


Version 0.12

## Introduction

This Jupyter Notebook uses the Ansible builtin `uri` method for getting and setting parameters via the Redfish service of an iLO 5.

## <a id = "EnvironmentPreparation">Environment preparation</a>

The following cell sets environment variables and checks the connectivity toward the various BMCs used in this notebook.

In [ ]:
########## Environment preparation (Version: 0.1) ############

# Set Student ID number
stdid={{ STDID }}
Id=$(id --user --name)

# location and ports variables
IloSyBasePort={{ ILOSYBASEPORT }}
let iLO5SimulatorBasePort=$IloSyBasePort
let iLO5SimulatorPort=${iLO5SimulatorBasePort}+${stdid}
let ilo5Port=443

iLO5SimulatorIP=ilo5simulators
iLOSimulator=${iLO5SimulatorIP}:${iLO5SimulatorPort}
iLO5SimulatorURI=https://${iLOSimulator}

ilo5IP="iLO5SimulatorIP"
ilo5="${ilo5IP}:${ilo5Port}"
ilo5URI="https://${ilo5}"

# Credentials
User="student"
Password='P@ssw0rd!'

# Miscellaneous
#alias ResetIlo5Simulator="ssh rstsimul@${iLO5SimulatorIP} $Id $iLO5SimulatorBasePort"
w=$(basename $PWD)
alias ResetSimulators="../create-globalbmc.shc.x &>/dev/null"

# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in iLO5Simulator ; do
    ip="${bmc}IP" ; port=$(echo ${bmc}Port)
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &>/dev/null && 
        echo "$bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done


# Create Ansible inventory file
cat >> hosts << __EOF__
[OneViewManagedBmcs]
ilo5simulators ansible_port={{ ILOSYBASEPORT }}

__EOF__

### <a id = "iLO5SimulatorRestart">Restart iLO 5 simulator</a>

If you need or desire to restart your iLO 5 simulator to restart this workshop from scratch or for other reasons, run the following cell at any time.

In [ ]:
# iLO 5 Simulator restart
ResetIlo5Simulator 

# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in ilo5 OpenBmc iLO5Simulator ; do
    ip="${bmc}IP" ; port=$(echo ${bmc}Port)
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &> /dev/null && 
        echo "$bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done

## Get and Set Redfish properties using Ansible built-in uri module 

The following cells retrieves properties from hosts listed in the [hosts](hosts) file using the [ansible.builtin.uri](https://docs.ansible.com/ansible/latest/collections/ansible/builtin/uri_module.html) module.

### GET SSD Wear status

In [ ]:
ansible-playbook -i hosts 2/GetWearStatusUsingBuiltInUri.yml

### Change IndicatorLED status

TBD

In [ ]:
ansible-playbook -i hosts 2/SetIndicatorLEDUsingBuilInUri.yml

# The following cells are not part of the Ansible Redfish Notebook

## Just examples from the WKSHP-iloREST workshop

### <a id = "IndicatorLED">Indicator LED</a>

The chassis indicator LED corresponds to the blue LED that is used to identify the chassis in a rack of the datacenter.

In [ ]:
# Select data type
ilorest select Chassis

# Get property value
IndicatorLED=$(ilorest get --json IndicatorLED | jq -r '.IndicatorLED')
echo "IndicatorLED=$IndicatorLED"

# Retrieve property info
ilorest info IndicatorLED

In [ ]:
# Refresh cache
ilorest select Chassis --refresh

# Change the indicator LED state
LEDState=$(ilorest get IndicatorLED | cut -d= --field=2)
echo -e "LED State:$LEDState\n"  

if [ "$LEDState" = "Off" ]; then
    LEDState="Lit"
else
    LEDState="Off"
fi

# Change property and commit
echo "Changing IndicatorLED to $LEDState"
ilorest set IndicatorLED="$LEDState" --commit
echo

# Validate property change
ilorest get IndicatorLED

### <a id = "ChassisAssetTag">Chassis Asset Tag</a>

The Chassis `AssetTag` is different from the `ComputerSystem` [Asset Tag](#ComputerSystemAssetTag). 

In [ ]:
# Select data type
ilorest select Chassis --refresh

# Get property value
ilorest get AssetTag

In [ ]:
# Change the Chassis Asset Tag property
ilorest set AssetTag="New Chassis Asset Tag" --commit

# Verify change
ilorest get AssetTag

<-- [Back to the list of examples](#BackToList) ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) -->

<-- [Back to the list of examples](#BackToList) ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) -->

<-- [Back to the list of examples](#BackToList) ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) -->

### <a id = "ManageBiosRbsuPassword">Bios/RBSU Login password</a>

In [ ]:
# Setting a password for the first time
ilorest setpassword FirstTimePassword '""'

# Removing the Bios/RBSU password
ilorest setpassword '""' FirstTimePassowrd

# No commit needed. No iLO nor system reset needed. 
ilorest status


<-- [Back to the list of examples](#BackToList) ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) -->

### <a id = "ComputerSystemAssetTag">ComputerSystem Asset Tag</a>

The `ComputerSystem` Asset Tag is different from the `Chassis` [Asset Tag](#ChassisAssetTag).

In [ ]:
# Select data type and clear cache
ilorest select ComputerSystem. --refresh

# Retrieve ComputerSystem Asset Tag
ilorest get AssetTag

# Change ComputerSystem Asset Tag
ilorest set AssetTag="New ComputerSystem Asset Tag" --commit

# Verify change
ilorest get AssetTag

<-- [Back to the list of examples](#BackToList) ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) -->

## Security

### <a id = "SecurityDashboard">Security Dashboard</a>

The following five examples retrieve different security dashboard properties.

#### Overall security dashboard status

Use keyword `Security Dashboard` or `SecurityDashboard` in the [API reference document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#hpeilosecuritydashboard-v1_0_0-hpeilosecuritydashboard) to find the overall security dashboard data type.

In [ ]:
# Select data type
ilorest select HpeiLOSecurityDashboard.

# Retrieve overall security dashboard status 
ilorest get 


#### All Security parameters and properties in json format

Use keyword `SecurityParam` to find the corresponding data type in the [API Reference Document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#hpeilosecurityparam-v1_1_0-hpeilosecurityparam)

In [ ]:
# Select data type
ilorest select HpeiLOSecurityParam.

# List security params in JSON format
ilorest list --json

#### List all security params at risk

To retrieve this list you don't need to select the data type as it is already selected.

In [ ]:
ilorest list --filter SecurityStatus="Risk"

#### Get specific security parameter properties

In [ ]:
ilorest get --filter Name="Require Login for iLO RBSU"

#### Get specific property of a specific security parameter

In [ ]:
ilorest get Ignore --filter Name="Minimum Password Length"

<-- [Back to the list of examples](#BackToList) ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) -->

### <a id = "iLOSecurityMode">Security Mode/State</a>

From the iLO 5 Graphical User Interface, in the `Security` --> `Encryption` menu, you can view and modify the `SecurityState` setting. This setting is part of the `HpeSecurityService` data type as mentioned in the [API Reference Document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#securitystate)

>NOTE: Software like the Smart Update Tool (`SUT`) may refer to this setting as `Security Mode` instead of `Security State`. 

#### Retrieve and modify the iLO 5 Security State setting

The following cell retrieves all the properties of the `HpeSecurityService` and modifies the `SecurityState`. A similar change on a real iLO 5 triggers **automatically a reset** of the iLO. 

In [ ]:
# Select data type and refresh cache
ilorest select HpeSecurityService. --refresh

# Retrieve all Security Service paramters
ilorest get --json

# Modify Security State
SecurityState=$(ilorest get SecurityState | cut --delimiter== --field=2)
echo 

if [ "$SecurityState" = "Production" ]; then
   echo "Setting Security State to HighSecurity"
   ilorest set SecurityState="HighSecurity" --commit
else
    echo "Setting Security State to Production"
    ilorest set SecurityState="Production" --commit
fi

echo
ilorest get SecurityState 

<-- [Back to the list of examples](#BackToList) ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) -->

### <a id = "LoginSecurityBanner">Login Security Banner</a>

From the iLO 5 Graphical User Interface, in the `Security` --> `Login Security Banner` menu, you can enable/disable and customize the `LoginSecurityBanner` setting. This setting is part of the `HpeSecurityService` data type as mentioned in the [API Reference Document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#loginsecuritybanner)

#### Retrieve and modify the iLO 5 Security State setting

The following cell modifies the state of the Security Login Banner and provides a message when enabled. You don't need to reset the iLO 5 to effectively modify this banner.

In [ ]:
# Select data type and refresh cache
ilorest select HpeSecurityService. --refresh

# Retrieve Login Security Banner settings
ilorest get LoginSecurityBanner --json

# Modify Login Security Banner settings
LoginSecurityBanner=$(ilorest get LoginSecurityBanner/IsEnabled | cut --delimiter== --field=2)
echo 

if [ $LoginSecurityBanner = False ]; then
   echo "Enabling Login Security Banner with a custom message"
   ilorest set LoginSecurityBanner/IsEnabled=True --commit
   ilorest set LoginSecurityBanner/SecurityMessage="This is My Custom Security Message" --commit
else
    echo "Disabling Login Security Banner"
    # The following set command is not required with a real iLO 5. 
    # It's only there to get a "realistic" behavior of the simulator.
    ilorest set LoginSecurityBanner/SecurityMessage="" --commit
    ilorest set LoginSecurityBanner/IsEnabled=False --commit
fi

echo
ilorest get --json LoginSecurityBanner 

<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList) ; [Logout](#Logout) ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) -->

## Bios

### <a id = "BiosSerialNumber">Server Serial Number stored in BIOS</a>

In a computer node, the Serial Number stored in the `Bios.` data type corresponds to the server mother board serial number. HPE requires to change this serial number in case of mother board replacement to match the chassis serial number. 

In [ ]:
# Select data type
ilorest select Bios. --refresh
echo

# Make sure SN is read-write
ilorest info Attributes/SerialNumber --latestschema
echo

# Get SerialNumber from Bios data type.
ilorest get Attributes/SerialNumber

echo "Set new SN"
ilorest set Attributes/SerialNumber="NEWSN" --commit

# Reload a fresh simulator cache
ilorest select Bios. --refresh
echo 

# Validate the SN change
ilorest get Attributes/SerialNumber


<-- [Back to the list of examples](#BackToList) ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) -->

## Summary

In this workshop, you discovered several HPE iLOrest examples. Go to the [Conclusion Notebook](3-Conclusion.ipynb) if you are finished.